# LangChain basics

docs: [ChatOpenAI](https://python.langchain.com/docs/how_to/structured_output/)

See [README.md](/README.md) for how to set up environment.


In [ ]:
import os
import json

from langchain_openai import ChatOpenAI

# Client & Model
client = ChatOpenAI(
    model="gpt-4o-mini",
    base_url=os.environ["OPENAI_BASE_URL"],
    api_key=lambda: os.environ["OPENAI_API_KEY"]
)


___
### From pydantic.BaseModel

In [ ]:
from pydantic import BaseModel, Field

class Schema(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: int = Field(description="How funny the joke is, from 1 to 10")


client_with_structured_output = client.with_structured_output(Schema)
response: Schema = client_with_structured_output.invoke("Tell me a joke about cats")  # type: ignore
print(f"setup: {response.setup}")
print(f"punchline: {response.punchline}")
print(f"rating: {response.rating}")


___
### From dict

In [ ]:
schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}

client_with_structured_output = client.with_structured_output(schema)
response = client_with_structured_output.invoke("Tell me a joke about cats")
print(json.dumps(response, indent=2))


In [ ]:
# NOTE: To make a property non-generative: set default value, remove its 'description' and add it to required
schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            # 'description': 'How funny the joke is, from 1 to 10',
            "default": 5,
        },
    },
    "required": ["setup", "punchline", "rating"],
}

client_with_structured_output = client.with_structured_output(schema)
response = client_with_structured_output.invoke("Tell me a joke about cats")
print(json.dumps(response, indent=2))
